# Multi Train Gradient Update

## Importing Libraries

In [1]:
from typing import Dict
import threading

import gym
import numpy as np
import torch as th

from stable_baselines3 import A2C as ALGO
from stable_baselines3.common.evaluation import evaluate_policy

## Init. ENV and Model

In [2]:
env = gym.make('CartPole-v1')
model = ALGO(
    "MlpPolicy",
    env
)

model_trained_1 = ALGO(
    "MlpPolicy",
    env
)

model_trained_2 = ALGO(
    "MlpPolicy",
    env
)

## Functions to Evaluate Model and Train Model within Thread

In [3]:
def evaluate(model, env, message = '', verbose = 0):
    fitnesses = []
    iterations = 10
    for i in range(iterations):
        fitness, _ = evaluate_policy(model, env)
        if verbose == 1:
            print(i, fitness, end=" ")
        fitnesses.append(fitness)

    mean_fitness = np.mean(sorted(fitnesses))
    print(f'Type {message} Mean reward: {mean_fitness}')

In [4]:
def train(model, timesteps):
    print('Starting Training')
    model.learn(total_timesteps=timesteps)
    print('Completed Training')

## Initial Evaluation

In [5]:
evaluate(model, env)
evaluate(model_trained_1, env)
evaluate(model_trained_2, env)

/home/arnab/.miniconda3/envs/pydrl/lib/python3.7/site-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


Type  Mean reward: 63.64
Type  Mean reward: 9.45
Type  Mean reward: 8.92


## Train for 1K Steps and Evaluate

In [6]:
# Train MT Model 1
t1 = threading.Thread(target=train, args=(model_trained_1, 10_00))

# Train MT Model 2
t2 = threading.Thread(target=train, args=(model_trained_2, 10_00))

# starting thread
t1.start()
t2.start()

# wait until thread is completely executed
t1.join()
t2.join()


# model_trained.learn(total_timesteps=10_00)
evaluate(model_trained_1, env)
evaluate(model_trained_2, env)
evaluate(model, env)

Starting Training
Starting Training
Completed Training
Completed Training
Type  Mean reward: 73.7
Type  Mean reward: 9.319999999999999
Type  Mean reward: 64.47


## Apply Gradient and Evaluate

In [7]:
# For Trained Model 1
state_dict = model.policy.state_dict()
optim_dict = model_trained_1.policy.optimizer.param_groups[0]['params']
optim_alpha = model.policy.optimizer.param_groups[0]['alpha']

optim_index = 0
for key, value in state_dict.items():
    # print(key)
    state_dict[key].add_(optim_alpha, optim_dict[optim_index])
    optim_index += 1

model.policy.load_state_dict(state_dict)

# For Trained Model 2
state_dict = model.policy.state_dict()
optim_dict = model_trained_2.policy.optimizer.param_groups[0]['params']
optim_alpha = model.policy.optimizer.param_groups[0]['alpha']

optim_index = 0
for key, value in state_dict.items():
    # print(key)
    state_dict[key].add_(optim_alpha, optim_dict[optim_index])
    optim_index += 1

model.policy.load_state_dict(state_dict)


evaluate(model, env)

/home/arnab/.miniconda3/envs/pydrl/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1025.)
  if __name__ == '__main__':


Type  Mean reward: 88.98


In [8]:
for i in range(10):
    print('Train Iter: ', i)

    # Train MT Model 1
    t1 = threading.Thread(target=train, args=(model_trained_1, 10_00))

    # Train MT Model 2
    t2 = threading.Thread(target=train, args=(model_trained_2, 10_00))

    # starting thread
    t1.start()
    t2.start()

    # wait until thread is completely executed
    t1.join()
    t2.join()

    evaluate(model_trained_1, env, 'Trained Model 1', verbose=1)
    evaluate(model_trained_2, env, 'Trained Model 2', verbose=1)
    evaluate(model, env, 'Initial Model', verbose=1)

    # For Trained Model 1
    state_dict = model.policy.state_dict()
    optim_dict = model_trained_1.policy.optimizer.param_groups[0]['params']
    optim_alpha = model.policy.optimizer.param_groups[0]['alpha']

    optim_index = 0
    for key, value in state_dict.items():
        # print(key)
        state_dict[key].add_(optim_alpha, optim_dict[optim_index])
        optim_index += 1

    model.policy.load_state_dict(state_dict)

    # For Trained Model 2
    state_dict = model.policy.state_dict()
    optim_dict = model_trained_2.policy.optimizer.param_groups[0]['params']
    optim_alpha = model.policy.optimizer.param_groups[0]['alpha']

    optim_index = 0
    for key, value in state_dict.items():
        # print(key)
        state_dict[key].add_(optim_alpha, optim_dict[optim_index])
        optim_index += 1

    model.policy.load_state_dict(state_dict)


    evaluate(model, env, 'Updated Model', verbose=1)

Train Iter:  0
Starting Training
Starting Training
Completed Training
Completed Training
0 263.3 1 222.7 2 308.9 3 166.0 4 251.6 5 337.3 6 205.5 7 203.8 8 183.6 9 177.2 Type Trained Model 1 Mean reward: 231.99
0 116.9 1 111.6 2 129.6 3 110.9 4 103.6 5 121.7 6 134.1 7 109.5 8 152.0 9 108.4 Type Trained Model 2 Mean reward: 119.83
0 105.7 1 120.0 2 99.4 3 96.9 4 154.0 5 108.6 6 76.0 7 78.4 8 91.5 9 123.5 Type Initial Model Mean reward: 105.4
0 142.7 1 117.9 2 153.0 3 142.9 4 165.4 5 127.2 6 126.5 7 164.6 8 115.3 9 139.9 Type Updated Model Mean reward: 139.54000000000002
Train Iter:  1
Starting Training
Starting Training
Completed Training
Completed Training
0 259.8 1 187.3 2 274.6 3 171.1 4 231.1 5 305.7 6 365.7 7 120.8 8 204.0 9 157.6 Type Trained Model 1 Mean reward: 227.77000000000004
0 195.0 1 168.0 2 175.5 3 206.5 4 203.2 5 171.2 6 168.4 7 160.5 8 179.7 9 197.5 Type Trained Model 2 Mean reward: 182.55
0 112.9 1 126.5 2 174.3 3 95.1 4 153.2 5 109.3 6 153.8 7 98.4 8 106.9 9 209.6 Type

In [9]:
model.get_parameters()

{'policy': OrderedDict([('mlp_extractor.policy_net.0.weight',
               tensor([[-4.7994,  2.7285,  1.8746,  4.0797],
                       [ 0.7623, -7.0949,  2.1134, -1.0655],
                       [ 1.9194,  3.3921, -0.3741,  3.4626],
                       [-3.6278, -1.0268, -2.4102,  3.2928],
                       [ 1.1961, -0.9835, -2.6474,  4.2740],
                       [-3.1046,  3.6106, -5.5946,  0.2108],
                       [ 0.1898, -4.1656, -1.2745,  1.0092],
                       [-3.5225,  0.8290,  4.2771,  2.9213],
                       [-0.1257, -0.5160, -1.3684, -3.4648],
                       [-1.7615,  0.8610,  1.5810,  3.8147],
                       [-3.9144, -0.5135,  0.4511,  1.2394],
                       [ 4.6794,  1.8337, -0.4528, -0.6097],
                       [ 3.7276,  0.2791,  3.8379,  4.3595],
                       [ 1.5737, -0.0986, -0.5604, -1.5846],
                       [ 0.0509,  2.3838, -1.1136,  2.0262],
                       

In [10]:
model.save('a2c_lunar_multiproc')

In [11]:
# Exporting Params as JSON
## Function to Convert Params Dict to Flattened List
def flatten_list(params):
    """
    :param params: (dict)
    :return: (np.ndarray)
    """
    params_ = {}
    for key in params.keys():
        params_[key] = params[key].tolist()
    return params_
## Write Parameters to JSON File
import json

all_params = model.get_parameters()
pol_params = flatten_list(all_params['policy'])

all_params['policy'] = pol_params

with open('a2c_lunar_multiproc.json', 'w') as f:
    json.dump(all_params, f, indent='\t')

In [12]:
model_loaded = ALGO(
    "MlpPolicy",
    env
)

evaluate(model_loaded,env, verbose=1)

new_params = all_params
loaded_pol_params = new_params['policy']
for key in loaded_pol_params.keys():
    loaded_pol_params[key] = th.tensor(loaded_pol_params[key])

new_params['policy'] = loaded_pol_params

model_loaded.set_parameters(new_params)

0 285.4 1 150.9 2 149.6 3 153.2 4 146.2 5 235.6 6 233.5 7 151.5 8 243.4 9 287.2 Type  Mean reward: 203.65000000000003


In [13]:
env.reset()
evaluate(model_loaded,env, verbose=1)

0 153.9 1 128.9 2 182.4 3 145.9 4 161.4 5 159.1 6 152.4 7 185.1 8 177.8 9 145.6 Type  Mean reward: 159.25
